##                         **CLOUD-BASED PE MALWARE DETECTION API**

In [0]:
!wget https://pubdata.endgame.com/ember/ember_dataset_2017_2.tar.bz2 --no-check-certificate

--2020-04-16 00:03:04--  https://pubdata.endgame.com/ember/ember_dataset_2017_2.tar.bz2
Resolving pubdata.endgame.com (pubdata.endgame.com)... 64.250.189.21
Connecting to pubdata.endgame.com (pubdata.endgame.com)|64.250.189.21|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 1751237573 (1.6G) [application/octet-stream]
Saving to: ‘ember_dataset_2017_2.tar.bz2’

ember_dataset_2017_ 100%[===================>]   1.63G  13.7MB/s    in 61s     

2020-04-16 00:04:05 (27.3 MB/s) - ‘ember_dataset_2017_2.tar.bz2’ saved [1751237573/1751237573]



In [0]:
!tar -xvf ember_dataset_2017_2.tar.bz2

ember_2017_2/
ember_2017_2/train_features_1.jsonl
ember_2017_2/train_features_0.jsonl
ember_2017_2/train_features_3.jsonl
ember_2017_2/test_features.jsonl
ember_2017_2/train_features_5.jsonl
ember_2017_2/train_features_4.jsonl
ember_2017_2/train_features_2.jsonl


In [0]:
!wget https://github.com/endgameinc/ember/archive/master.zip
!unzip master.zip
!rm master.zip
!cp -r ember-master/* .
!rm -r ember-master

--2020-04-15 01:45:07--  https://github.com/endgameinc/ember/archive/master.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/endgameinc/ember/zip/master [following]
--2020-04-15 01:45:07--  https://codeload.github.com/endgameinc/ember/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [      <=>           ]  11.22M  4.90MB/s    in 2.3s    

2020-04-15 01:45:11 (4.90 MB/s) - ‘master.zip’ saved [11769324]

Archive:  master.zip
f9a018632ba108b4e25c33d6c3e2e7a7c4487f58
   creating: ember-master/
  inflating: ember-master/LICENSE.txt  
  inflating: ember-master/README.md  
   creating:

In [0]:
!pip install -r requirements.txt
!python setup.py install

     |████████████████████████████████| 3.5MB 2.7MB/s 
running install
running bdist_egg
running egg_info
creating ember.egg-info
writing ember.egg-info/PKG-INFO
writing dependency_links to ember.egg-info/dependency_links.txt
writing requirements to ember.egg-info/requires.txt
writing top-level names to ember.egg-info/top_level.txt
writing manifest file 'ember.egg-info/SOURCES.txt'
reading manifest file 'ember.egg-info/SOURCES.txt'
writing manifest file 'ember.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/ember
copying ember/features.py -> build/lib/ember
copying ember/__init__.py -> build/lib/ember
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/ember
copying build/lib/ember/features.py -> build/bdist.linux-x86_64/egg/ember
copying build/lib/ember/__init__.py -> build/bdist.linux-x86_64/egg/ember
byte-co

#vectorizing Sample Features

In [0]:
import ember
ember.create_vectorized_features("ember_2017_2/")
ember.create_metadata("ember_2017_2/")

Vectorizing training set


100%|██████████| 900000/900000 [31:17<00:00, 479.34it/s]


Vectorizing test set


100%|██████████| 200000/200000 [06:54<00:00, 482.77it/s]


,sha256,appeared,subset,label
0,0abb4fda7d5b13801d63bee53e5e256be43e141faa077a...,2006-12,train,0
1,d4206650743b3d519106dea10a38a55c30467c3d9f7875...,2006-12,train,0
2,c9cafff8a596ba8a80bafb4ba8ae6f2ef3329d95b85f15...,2007-01,train,0
3,7f513818bcc276c531af2e641c597744da807e21cc1160...,2007-02,train,0
4,ca65e1c387a4cc9e7d8a8ce12bf1bcf9f534c9032b9d95...,2007-02,train,0
...,...,...,...,...
1099995,fffe314f23cee3a68ccab272934877d3bc18ec3bd905df...,2017-12,test,0
1099996,fffe7a1b23e04facc9ca91a93ac4a34e8b3040e023dbde...,2017-12,test,1
1099997,fffe801f51e7ec931515aa49a3d157a9c0fbcdca8c9d80...,2017-12,test,0
1099998,fffe92f9593649c4a7050302368189de45e2c1c06b04ea...,2017-12,test,1


#Building Model

In [0]:
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import layers
  feature_size=2381
  tf.compat.v1.disable_eager_execution()

  keras.backend.clear_session()
  
  #Model architecture
  from tensorflow.keras import layers
  
  model = tf.keras.Sequential()
  model.add(layers.InputLayer(input_shape=(1,feature_size)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1000, activation='relu'))
  model.add(layers.Dense(1800, activation='relu'))
  model.add(layers.Dense(2000, activation='relu'))


  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 1, 2381)           0         
_________________________________________________________________
dense (Dense)                (None, 1, 1000)           2382000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1000)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              1001      
Total params: 2,383,001
Trainable params: 2,383,001
Non-trainable params: 0
_________________________________________________________________


In [0]:
import ember
X_train, y_train, X_test, y_test = ember.read_vectorized_features("ember_2017_2/")
metadata_dataframe = ember.read_metadata("ember_2017_2/")

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
labelrows = (y_train != -1)

In [0]:
X_train = X_train[labelrows]
y_train = y_train[labelrows]

In [0]:
#Standardizing the training data
from sklearn.preprocessing import StandardScaler
std = StandardScaler()

for x in range(0,700000,100000):
  std.partial_fit(X_train[x:x+100000])

In [0]:
X_train = std.fit_transform(X_train)

In [0]:
# reshaping the training data
import numpy as np
X_train = np.reshape(X_train,(-1,1,2381))
y_train = np.reshape(y_train,(-1,1,1))

In [0]:
#Standardizing the testing data
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()

X_test = mms.fit_transform(X_test)

In [0]:
X_test = std.fit_transform(X_test)

In [0]:
# reshaping the testing data
import numpy as np
X_test = np.reshape(X_test,(-1,1,2381))
y_test = np.reshape(y_test,(-1,1,1))

#Model Training and Evaluating

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


In [0]:
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='binary_crossentropy',
          metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])

In [0]:
history = model.fit(X_train, y_train,
                batch_size=315,
                epochs=1,
                  validation_data =((X_test,y_test))
                  )


Train on 700000 samples, validate on 200000 samples
700000/700000 [==============================] - 179s 298us/sample - loss: 4.2120 - accuracy: 0.9646 - val_loss: 3.3853 - val_accuracy: 0.9303


#prediction

In [0]:
prediction= model.predict(X_test)

In [0]:
y_test = np.reshape(y_test,(-1))
prediction = np.reshape(prediction,(-1))

In [0]:
y_test

(array([1., 1., 0., ..., 0., 1., 1.], dtype=float32)


In [0]:
prediction

(array([1., 1., 0., ..., 0., 1., 1.], dtype=float32))


#F1 Score

In [0]:
from sklearn.metrics import f1_score
score = f1_score(y_test_int, pred_int) 

In [0]:
score

0.9367284


#Precision

In [0]:
precision = sklearn.metrics.precision_score(y_test_int,pred_int)
precision

0.963842

In [0]:
!wget https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe   

--2020-04-16 04:04:38--  https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe
Resolving the.earth.li (the.earth.li)... 93.93.131.124, 2a00:1098:86:4d:c0ff:ee:15:900d
Connecting to the.earth.li (the.earth.li)|93.93.131.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://the.earth.li/~sgtatham/putty/0.73/w32/putty.exe [following]
--2020-04-16 04:04:38--  https://the.earth.li/~sgtatham/putty/0.73/w32/putty.exe
Reusing existing connection to the.earth.li:443.
HTTP request sent, awaiting response... 200 OK
Length: 1096080 (1.0M) [application/x-msdos-program]
Saving to: ‘putty.exe.1’

putty.exe.1         100%[===================>]   1.04M   520KB/s    in 2.1s    

2020-04-16 04:04:40 (520 KB/s) - ‘putty.exe.1’ saved [1096080/1096080]



In [0]:
from sklearn.preprocessing import StandardScaler
std_1 = StandardScaler()

In [0]:

def testing(pefile):
  try:
    import ember
  except:
    !wget https://github.com/endgameinc/ember/archive/master.zip
    !unzip master.zip
    !rm master.zip
    !cp -r ember-master/* .
    !rm -r ember-master
    !pip install -r requirements.txt
    !python setup.py install
    import ember

  from sklearn.preprocessing import StandardScaler
  import tensorflow as tf
  tf.compat.v1.disable_eager_execution()
  data = open(pefile, "rb").read()
  features = ember.PEFeatureExtractor(2)
  data = np.array(extractor.feature_vector(data), dtype=np.float32)
  data = std_1.transform([data])
  data = np.reshape(data,(-1,1,2381))
  prediction = model.predict_classes(data)

  return prediction

In [0]:
testing("putty.exe")

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
array([[[0]]], dtype=int32)
